**Table-aware plot embedding**

It should be possible to embed a bar plot or a line plot more easily than the current approach:

* The data for the plot would come from table rows / columns, instead of separately-supplied series.
    * This would eliminate the awkward "reversal" of series data.
* The layout of the embedded plot should be responsive to cell spacing, gaps, padding, etc.
    * This would eliminate awkward manual padding to align vis datums with cells.
* The new plotting code replaces the current merge + create axes approach:
    
            # Instead of this:
            axes = <cells>.merge().axes(...)
            axes.bars(data, ...)
            
            # Do this:
            <cells>.bars() # Uses the data already in the table cells.

* The axes and mark are constructed lazily in toyplot.coordinates.Table.finalize().
* The main challenge is overriding the current code to create projections.
    * The table code needs to specify an explicit projection based on the table layout, instead of "log", "linear", etc.

**Odds-and-ends**

* Do a better job updating gaps when we insert / delete cells.
* Should grid lines be rendered down the middle of a gap (as we do today), or to either side?

In [1]:
import numpy
import toyplot.data

import logging
toyplot.log.setLevel(logging.DEBUG)

In [4]:
canvas, table = toyplot.table(trows=3, rows=3, brows=3, lcolumns=3, columns=3, rcolumns=3, filename="rainbow")
table.cells.cells.data = numpy.arange(81).reshape(9, 9)
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
table.top.left.cells.style = {"fill":"red", "opacity":0.3}
table.top.cells.style = {"fill":"orange", "opacity":0.3}
table.top.right.cells.style = {"fill":"yellow", "opacity":0.3}
table.right.cells.style = {"fill":"greenyellow", "opacity":0.3}
table.bottom.right.cells.style = {"fill":"green", "opacity":0.3}
table.bottom.cells.style = {"fill":"aqua", "opacity":0.3}
table.bottom.left.cells.style = {"fill":"blue", "opacity":0.3}
table.left.cells.style = {"fill":"purple", "opacity":0.3}

numpy.random.seed(1234)
axes = table.left.column[1].merge().cartesian(cell_padding=5)
axes.bars(numpy.random.normal(loc=5, size=(20, 2)), along="y", opacity=0.5);
axes = table.right.column[1].merge().cartesian(cell_padding=5)
axes.plot(numpy.random.normal(loc=5, size=(20, 3)), along="y");
table.cells.column[4].bars()
#table.cells.row[0:3].height = 25
#table.cells.row[4].height = "1in"
#table.cells.column[4].width = "1in"
#table.cells.gaps.rows[7] = 10

0 1 2 3 5 6 7 8 9 10 11 12 14 15 16 17 18 19 20 21 23 24 25 26 27 29 30 32 33 35 36 38 39 41 42 44 45 47 48 50 51 53 54 55 56 57 59 60 61 62 63 64 65 66 68 69 70 71 72 73 74 75 77 78 79 80 4 <path d="M 473.52881213342243 378.33333333333331 L 456.01076336237247 370.08771929824564 L 471.79429826972671 361.84210526315786 L 474.66131642791925 353.59649122807014 L 461.4913544310956 345.35087719298247 L 471.7305782559169 337.10526315789474 L 477.51535642504359 328.85964912280701 L 472.33543155200812 320.61403508771929 L 481.32700916054523 312.36842105263156 L 463.60329839221697 304.12280701754389 L 466.51211424227006 295.87719298245617 L 477.17145489633447 287.63157894736844 L 474.75654585787549 279.38596491228071 L 473.23005342264554 271.14035087719299 L 483.66666810537919 262.89473684210532 L 469.40581127958228 254.64912280701756 L 475.11382685504293 246.40350877192986 L 469.65118309761453 238.15789473684211 L 479.5581918630491 229.91228070175441 L 477.21038213177417 221.66666666666669" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 468.29544707537161 378.33333333333331 L 474.92617157003497 370.08771929824564 L 473.05722050655015 361.84210526315786 L 479.7570136325262 353.59649122807014 L 472.08645858802134 345.35087719298247 L 475.79123727222509 337.10526315789474 L 485.4953104600387 328.85964912280701 L 475.41621841089034 320.61403508771929 L 453.36025876774761 312.36842105263156 L 474.63374919762941 304.12280701754389 L 476.67105301935533 295.87719298245617 L 465.36535798908619 287.63157894736844 L 463.52163713029881 279.38596491228071 L 476.69004838088 271.14035087719299 L 474.15122546531597 262.89473684210532 L 479.15732795573473 254.64912280701756 L 484.24388202737947 246.40350877192986 L 464.5380119261888 238.15789473684211 L 472.24094958012586 229.91228070175441 L 464.20300399089967 221.66666666666669" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 473.20260381813466 378.33333333333331 L 465.60231286962482 370.08771929824564 L 479.05997477003575 361.84210526315786 L 461.13437076355183 353.59649122807014 L 468.44368659489305 345.35087719298247 L 472.61908372270216 337.10526315789474 L 464.97461780500015 328.85964912280701 L 471.21037940577094 320.61403508771929 L 489.4444444444444 312.36842105263156 L 478.64715488519874 304.12280701754389 L 478.64314504946719 295.87719298245617 L 489.25909359082186 287.63157894736844 L 478.05478921696727 279.38596491228071 L 443.88888888888886 271.14035087719299 L 474.24812454969111 262.89473684210532 L 480.92894905147148 254.64912280701756 L 473.55847655974316 246.40350877192986 L 468.14664728898174 238.15789473684211 L 470.10069101550152 229.91228070175441 L 468.73979247977314 221.66666666666669" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> 0 25 50 75 
 
 
 Save as .csv

In [3]:
import toyplot.pdf
toyplot.pdf.render(canvas, "test.pdf")